# Modelo de clasificación de imágenes de lengua de señas en Español

## Creación del modelo

### Librerías

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
import cv2
from tensorflow.keras.models import load_model
import json
from tensorflow.keras.preprocessing import image

### (Opcional) Redimensionar imágenes 

In [15]:
# import cv2

# input_folder = "./classes_original/"
# output_folder = "./classes/"
# new_size = (224, 224)

# for root, dirs, files in os.walk(input_folder):
#     relative_path = os.path.relpath(root, input_folder)
#     output_subfolder = os.path.join(output_folder, relative_path)
#     os.makedirs(output_subfolder, exist_ok=True)
#     for file in files:
#         input_path = os.path.join(root, file)
#         output_path = os.path.join(output_subfolder, file)
#         img = cv2.imread(input_path)
#         resized_img = cv2.resize(img, new_size)
#         cv2.imwrite(output_path, resized_img)

### Data Augmentation

In [2]:


data_path = "./classes_poses/"

def load_images(path):
    images = []
    labels = []
    classes = os.listdir(path)
    class_count = len(classes)

    for class_name in classes:
        class_path = os.path.join(path, class_name)
        for file_name in os.listdir(class_path):
            img_path = os.path.join(class_path, file_name)

            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

            images.append(img)
            labels.append(class_name)

    return np.array(images), np.array(labels), class_count

X, y, class_count = load_images(data_path)

# Imprimir el número de muestras por clase antes del balanceo
print("Número de muestras por clase antes del balanceo:")
for class_name in set(y):
    print(f"{class_name}: {np.sum(y == class_name)} muestras")

# Aplicar oversampling a las clases menos representadas
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X.reshape(-1, 224 * 224 * 3), y)

X_resampled = X_resampled.reshape(-1, 224, 224, 3)

# Crear un generador de imágenes con data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Aplicar data augmentation al conjunto de datos resampleado
augmented_images = []
augmented_labels = []

for img, label in zip(X_resampled, y_resampled):
    img = np.expand_dims(img, axis=0)
    for _ in range(3):  # Aumentar el conjunto de datos tres veces
        augmented_img = datagen.random_transform(img[0])
        augmented_images.append(augmented_img)
        augmented_labels.append(label)

X_ = np.array(augmented_images)
y_ = np.array(augmented_labels)


# Imprimir el número de muestras por clase después del balanceo
print("\nNúmero de muestras por clase después del balanceo:")
for class_name in set(y_):
    print(f"{class_name}: {np.sum(y_ == class_name)} muestras")

Número de muestras por clase antes del balanceo:
F: 102 muestras
E: 99 muestras
G: 97 muestras
M: 102 muestras
P: 90 muestras
C: 61 muestras
O: 93 muestras
A: 98 muestras
U: 100 muestras
I: 106 muestras
T: 93 muestras
R: 77 muestras
K: 96 muestras
B: 91 muestras
D: 81 muestras
N: 101 muestras
L: 107 muestras
Q: 85 muestras
S: 98 muestras

Número de muestras por clase después del balanceo:
F: 321 muestras
E: 321 muestras
G: 321 muestras
M: 321 muestras
P: 321 muestras
C: 321 muestras
O: 321 muestras
A: 321 muestras
U: 321 muestras
I: 321 muestras
T: 321 muestras
R: 321 muestras
K: 321 muestras
B: 321 muestras
D: 321 muestras
N: 321 muestras
L: 321 muestras
Q: 321 muestras
S: 321 muestras


### Partición de datos 

In [3]:
X_train, X_temp, y_train, y_temp = train_test_split(X_, y_, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
num_classes = len(np.unique(y_train))

### Transformación de variables categóricas a numéricas

In [4]:
import json
class_indices = {}
for i, class_name in enumerate(np.unique(y_train)):
    class_indices[class_name] = i
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

y_train = np.vectorize(class_indices.get)(y_train)
y_val = np.vectorize(class_indices.get)(y_val)
y_test = np.vectorize(class_indices.get)(y_test)


### Configurar el modelo de clasificación con CNN

In [5]:
def create_model(kernel_size, num_layers, batch_size, learning_rate):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (kernel_size, kernel_size), activation='relu', input_shape=(224, 224, 3)))
    for _ in range(num_layers - 1):
        model.add(tf.keras.layers.Conv2D(64, (kernel_size, kernel_size), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

### Entrenar modelo

In [6]:
# param_grid = {
#     'kernel_size': [3, 5],
#     'num_layers': [2, 3],
#     'batch_size': [32, 64],
#     'learning_rate': [0.001, 0.0001]
# }
# param_grid = {
#     'kernel_size': [5],
#     'num_layers': [3],
#     'batch_size': [32],
#     'learning_rate': [0.0001]
# }

# model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# grid_result = grid.fit(X_train, y_train)

# best_params = grid_result.best_params_

 
best_params = {'batch_size': 32, 'kernel_size': 5, 'learning_rate': 0.0001, 'num_layers': 3}
final_model = create_model(**best_params)
final_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

y_pred = final_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))


Epoch 1/10
134/134 [==============================] - 1192s 9s/step - loss: 3.4680 - accuracy: 0.0703 - val_loss: 2.9000 - val_accuracy: 0.0918
Epoch 2/10
134/134 [==============================] - 1307s 10s/step - loss: 2.4352 - accuracy: 0.2959 - val_loss: 2.9841 - val_accuracy: 0.0973
Epoch 3/10
 37/134 [=======>......................] - ETA: 15:56 - loss: 0.6708 - accuracy: 0.8522

KeyboardInterrupt: 

### Guardar el modelo

In [ ]:
final_model.save('model.h5')


## Uso del modelo

### Detección de clase con una sola imágen

In [2]:

model = load_model('model.h5')

class_indices = json.load(open('class_indices.json'))

def predict_image(model, img_path, class_indices):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    prediction = model.predict(img)
    predicted_class = list(class_indices.keys())[np.argmax(prediction)]
    return predicted_class

img_path = "./classes_original/M/DSC01254.JPG"
predicted_class = predict_image(model, img_path, class_indices)
print(predicted_class)


1/1 [==============================] - 0s 372ms/step
M


### Detección de clases en tiempo real

In [ ]:
# quiero que me haga predicciones en tiempo real con la camara


model = load_model('model.h5')

class_indices = json.load(open('class_indices.json'))

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224, 224))
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    prediction = model.predict(img)
    predicted_class = list(class_indices.keys())[np.argmax(prediction)]
    cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()